In [43]:
import json
import pandas as pd
import os
site_to_zip_code = {}

with open('/vagrant/etl-natgrid-lime/test_data_hourly/maps/site_list.json') as fd:
    jsd = json.load(fd)
    dt = jsd['data']
    for dct in dt:
        site_id, zipcode = dct['site_id'], dct['s_postcode']
        site_to_zip_code[str(site_id)] = zipcode


# Loading the NATGrid hourly data
# 
import json
import numpy as np
import pytz
import pandas 
import os
from eemeter.weather.location import zipcode_to_usaf_station
from eemeter.weather import ISDWeatherSource
from datetime import datetime

#file_name = '/vagrant/etl-natgrid-lime/test_data_hourly/raw_site_data/2017-07-27/28610.json'
#file_name = '/vagrant/etl-natgrid-lime/test_data_hourly/raw_site_data/2017-07-27/28162.json'

format='%Y-%m-%d %H:%M:%S'

def prepare_hrl_data_md(file_name):
    with open(file_name) as fp:
        js_data = json.load(fp)
    
    dt = js_data['data']
    #print "Keys in the Json data ",js_data.keys()
    result = []
    index = []
    energy_consumed= []
    day_hrs = []
    week_day = []
    fname = os.path.basename(file_name)[:-5]
    zipcode = site_to_zip_code[fname]
    if not zipcode:
        print 'zipcode not found'
        return 
    for energy_data in dt:
        # energy_cs can have none values
        energy_cs = energy_data['energy_consumed']
        if energy_cs is None:
            continue
        energy_cs = float(energy_cs) * 0.001
        if not np.isfinite(energy_cs):
            print  energy_cs
        #print energy_cs
        time_st = energy_data['t_stamp']
        pd_dt_tm = pandas.to_datetime(time_st, format=format, utc=True)
        #day_hrs.append(str( pd_dt_tm.hour))
        #week_day.append(str(pd_dt_tm.dayofweek))
        index.append(pd_dt_tm)
        energy_consumed.append(energy_cs)
        
    dt_index = pandas.DatetimeIndex(index, tz=pytz.utc)
    df = pandas.DataFrame({'value' : np.array(energy_consumed, dtype=np.float16),
                           'project_id':[fname for xx in range(len(energy_consumed))],
                           'interpretation' : ['ELECTRICITY_CONSUMPTION_SUPPLIED' for xx in range(len(energy_consumed))]
                          }, 
                          
                          index=dt_index)
    df = df[~df.index.duplicated(keep='first')].sort_index()
    
    df = df.asfreq('H')
    if len(df) < 3:
        return None, None
    mid = df.index[len(df.index) / 2 ]
    end = df.index[(len(df.index) / 2 ) + 1]
    project_df = pandas.DataFrame({"project_id" : [fname],
                                   "zipcode" :  [zipcode],
                                   "project_start" :[ mid],
                                   "project_end" : [end]
                                   })
    
    return df, project_df


trace_df_lst = []
project_df_lst = []
for   file in os.listdir('/vagrant/etl-natgrid-lime/test_data_hourly/raw_site_data/2017-07-27/'):
    full_f = os.path.join('/vagrant/etl-natgrid-lime/test_data_hourly/raw_site_data/2017-07-27/', file)
    
    df_t, project_df_t = prepare_hrl_data_md(full_f)
    if df_t is None:
        continue
    trace_df_lst.append(df_t)
    project_df_lst.append(project_df_t)
    #print full_f


traces = pd.concat(trace_df_lst)
traces = traces[np.isfinite(traces['value'])]
projects = pd.concat(project_df_lst)

#print projects
#traces.to_csv("/vagrant/natgrid/traces.csv", index_label='start')
#projects.to_csv("/vagrant/natgrid/projects.csv", index=False)



In [61]:
import pandas as pd
def split_trace(trace_df, interv_start, intervention_end):
    baseline_period_trace = []
    reporting_period_trace = []
    all_trace = []
    baseline_index = []
    reporting_index = []
    all_trace_index = []
    for index, row in trace_df.iterrows():
        if index < interv_start:
            baseline_period_trace.append(row)
            baseline_index.append(index)
            all_trace.append(row)
            all_trace_index.append(index)
        elif index > intervention_end:
            reporting_period_trace.append(row)
            reporting_index.append(index)
            all_trace.append(row)
            all_trace_index.append(index)
    
    return (pd.DataFrame(baseline_period_trace, index=baseline_index), 
           pd.DataFrame(reporting_period_trace, index=reporting_index),
           pd.DataFrame(all_trace, index=all_trace_index))      
    
for index, project_df in enumerate(project_df_lst):
    trace_df = trace_df_lst[index]
    baseline_trace, reporting_trace, all_trace = split_trace(trace_df, project_df['project_start'][0], project_df['project_end'][0])
    #print baseline_trace, reporting_trace, all_trace
    baseline_trace = baseline_trace.asfreq('H')
    reporting_trace = reporting_trace.asfreq('H')
    all_trace = all_trace.asfreq('H')
                

                                             interpretation project_id  \
2017-06-01 00:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 01:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 02:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 03:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 04:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 05:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 06:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 07:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 08:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 09:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 10:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 11:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      27871   
2017-06-01 12:00:00+00:00  ELECTRICITY

                                             interpretation project_id  \
2017-06-01 00:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 01:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 02:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 03:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 04:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 05:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 06:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 07:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 08:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 09:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 10:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 11:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28149   
2017-06-01 12:00:00+00:00  ELECTRICITY

                                             interpretation project_id  \
2017-06-01 00:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 01:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 02:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 03:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 04:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 05:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 06:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 07:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 08:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 09:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 10:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 11:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28162   
2017-06-01 12:00:00+00:00  ELECTRICITY

                                             interpretation project_id  \
2017-06-01 00:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 01:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 02:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 03:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 04:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 05:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 06:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 07:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 08:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 09:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 10:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 11:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28190   
2017-06-01 12:00:00+00:00  ELECTRICITY

                                             interpretation project_id  \
2017-06-01 00:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 01:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 02:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 03:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 04:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 05:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 06:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 07:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 08:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 09:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 10:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 11:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28205   
2017-06-01 12:00:00+00:00  ELECTRICITY

...

                                             interpretation project_id  \
2017-06-19 11:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 12:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 13:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 14:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 15:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 16:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 17:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 18:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 19:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 20:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 21:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 22:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28787   
2017-06-19 23:00:00+00:00  ELECTRICITY

                                             interpretation project_id  \
2017-06-26 11:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 12:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 13:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 14:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 15:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 16:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 17:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 18:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 19:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 20:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 21:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 22:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      28891   
2017-06-26 23:00:00+00:00  ELECTRICITY

                                             interpretation project_id  \
2017-06-29 10:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 11:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 12:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 13:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 14:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 15:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 16:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 17:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 18:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 19:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 20:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 21:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29018   
2017-06-29 22:00:00+00:00  ELECTRICITY

                                             interpretation project_id  \
2017-07-06 11:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 12:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 13:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 14:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 15:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 16:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 17:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 18:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 19:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 20:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 21:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 22:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      29112   
2017-07-06 23:00:00+00:00  ELECTRICITY

                                             interpretation project_id  \
2017-07-12 12:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 13:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 14:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 15:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 16:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 17:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 18:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 19:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 20:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 21:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 22:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-12 23:00:00+00:00  ELECTRICITY_CONSUMPTION_SUPPLIED      30310   
2017-07-13 00:00:00+00:00  ELECTRICITY

In [ ]:
tt = traces[traces.index < ]